In [ ]:
# %pip install --upgrade pip setuptools
# %pip install selenium
# %pip install googletrans==4.0.0-rc1

In [ ]:
import os
import sys
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import asyncio
import json
import aiohttp
import openpyxl
import textwrap
#from understat import Understat
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd
from googletrans import Translator
from deep_translator import GoogleTranslator

In [ ]:
#explanation

In [ ]:
url='https://www.rest.co.il/kosher-restaurants/israel/kosher/' #creating url for primary pages
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64'}
response=requests.get(url,headers=header)
driver = webdriver.Chrome()
page_num=1
resturant_list=[]
infolistArr=[]
info_cnt=0
translator = Translator()
while(page_num<=1):#need 401 pages
    if response.status_code != 200:   # if maximum page exceeded, break out of loop
         break
    html_text=requests.get(url,headers=header)
    mainpage=html_text.text
    mainsoup = BeautifulSoup(mainpage,'html.parser') # turn primary page into soup
    rest_page=mainsoup.find_all("div",attrs={"class":"rest-title"})
    
    url_rest_page=[]
    
    for div in rest_page:
        href = div.a['href']
        url_rest_page.append("https://www.rest.co.il"+href)
    for url_rest in url_rest_page:
        html_rest_text=requests.get(url_rest,headers=header)
        restpage=html_rest_text.text
        restsoup = BeautifulSoup(restpage,'html.parser')
        rest_name=restsoup.find("div",attrs={"class":"reviews_info_box"})
        rest_name=rest_name.find('h1').text.strip()
        #print(rest_name)
        rest_add=restsoup.find("div", attrs={"class":"h5"}).text.strip()
        try:
            rest_open=restsoup.find("div", attrs={"class":"open_now_dropdown"}).text.strip()
        except AttributeError:
            rest_open='No Info'
        rest_type=restsoup.find("div", attrs={"class":"h6"}).text.strip()
        rest_rank=restsoup.find("div", attrs={"class":"reviews_box"})
        try:
            rest_rank=rest_rank.find("span").text.strip() 
        except AttributeError:
            rest_rank=0
        ratings=restsoup.find_all("div" ,attrs={"class" : "user_progress_widget"})
        name=[]
        rating=[]
        for x in ratings:
            name.append(x.find("span").text.strip())
            rating.append(x.find("strong").text.strip())
        reviews_num=restsoup.find("div",attrs={"class":"raviews_box_item"})
        print(name)
        try:
            reviews_num=reviews_num.find('a').text.strip().replace("חוות דעת",'') 
        except AttributeError:
            reviews_num=0
        driver.maximize_window()
        driver.get(url_rest)
        try:
            wait = WebDriverWait(driver, 10)
            moreinfo = wait.until(EC.visibility_of_element_located((By.XPATH, "//small[@data-event-id='76' and @data-action='open_tags_popup']")))
            moreinfo.click()
            infolist = driver.find_element(By.CLASS_NAME,'pop-scroll-wrap')
            #sleep(5)
            infolistArr.append(infolist.text.strip())
        except:
            infolistArr.append("None")
        print("///////////////////////////////////////////")
        
        temp=[]
        for i,x in enumerate(infolistArr):
            temp.append(infolistArr[i].replace('\n',','))
        new_array=[]
        for i,sub_array in enumerate(temp):
            # Create a temporary list to store the separated parts of each string
            temp_list = []
            sub_array=sub_array.split(',')
            new_array=[]            
            for sub in sub_array:
                new_array.append(f'{sub}')
        print(new_array)
        restaurant={'rest name':rest_name,'rest add':rest_add,'rest open':rest_open,'rest type':rest_type,'rest rank':rest_rank,'reviews num':reviews_num}

        for i,field in enumerate(name):
            if field is not None:
                translated = GoogleTranslator(source='auto', target='en').translate(field)
                if translated is not None:
                    restaurant.update({f'{translated} rank':rating[i]})
                    print(translated)
            else:
             print("Translation failed for:", field)

        for new in new_array:
            if new is not None:
                translated = GoogleTranslator(source='auto', target='en').translate(new)
                if translated is not None:
                    restaurant.update({translated:1})
                    print(translated)
            else:
             print("Translation failed for:", new)

        #print(restaurant)
        resturant_list.append(restaurant)
        info_cnt=info_cnt+1
    page_num=page_num+1
    url='https://www.rest.co.il/kosher-restaurants/israel/kosher/page-'+ str(page_num) + '/'
    #print(url)   
resturantdf=pd.DataFrame(resturant_list)

In [ ]:
# driver = webdriver.Edge()
# url = "https://easy.co.il/list/Restaurants?order=1" 
# cities = ["אשדוד"]#["רמלה","ראשון לציון", "תל אביב", "באר שבע" ,"אילת" ,"נהריה","חיפה","עכו","פתח תקווה","טבריה","ירושלים","הוד השרון","יבנה","הרצליה", "באר טוביה", "ראש פינה", "קרית אתא", "קרית ביאליק", "אלעד", "חד נס", "שוהם", "נתניה", "אשדוד"] 
# driver.get(url)

# for city in cities:
#     changelocation = driver.find_element(By.ID,'changeAddressButton')
#     changelocation.click()
#     sleep(5)
#     search_input = driver.find_element(By.ID,'SearchInputAddress')
#     search_input.send_keys(city + Keys.ENTER)
#     sleep(3)
#     while True:
#         try:
#             load_more_button = driver.find_element(By.CLASS_NAME,'next-page-button')
#             load_more_button.click()
#             sleep(2)
#         except:
#             break

#     Listrestaurants = driver.find_element(By.CLASS_NAME,'list-results')
#     elements = Listrestaurants.find_elements(By.CLASS_NAME, 'biz-name') 
#     for e in elements:
#         print(e.text)

#     driver.refresh()


In [ ]:
resturantdf.to_csv('Pull_Data.csv',encoding='utf-8-sig',index=False)

In [ ]:
resturantdf